In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/MyDrive/ECE570/Project/EfficientNet/efficientnet
!ls

/content/gdrive/MyDrive/ECE570/Project/EfficientNet/efficientnet
best_models	     Nets		  run_experiment.py
find_hyperparams.py  print_parameters.py  run_results_40_samples.ipynb
joe_trained_models   ray_results.txt	  trained_models
logs		     README.md		  trainingjoe.ipynb
main.py		     runexercises.ipynb


In [3]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!git add joe_trained_models
!git config --global user.email "pappas_joe@yahoo.com"
!git config --global user.name "Joe Pappas"
!git commit -m "more trained models"

!git push

[master e976d1f] more trained models
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 joe_trained_models/JoeNet_depth_scale_11_15_final_epoch_13.pt
Counting objects: 4, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 11.71 MiB | 9.92 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/joempappas/efficientnet.git
   daaf078..e976d1f  master -> master


In [3]:
import sys
sys.path.append('Nets/')

In [4]:
import Nets.JoeNet as jn
import math
import torch
from torch import nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torchvision
#from torchinfo import summary

In [5]:
import os
def save_model_all(model, save_dir, model_name, epoch):
    """
    :param model:  nn model
    :param save_dir: save model direction
    :param model_name:  model name
    :param epoch:  epoch
    :return:  None
    """
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_prefix = os.path.join(save_dir, model_name)
    save_path = '{}_epoch_{}.pt'.format(save_prefix, epoch)
    print("save all model to {}".format(save_path))
    output = open(save_path, mode="wb")
    torch.save(model.state_dict(), output)
    # torch.save(model.state_dict(), save_path)
    output.close() 

In [6]:
device = torch.device('cuda')
print(device)

cuda


In [13]:
from torchvision.transforms.functional import InterpolationMode
transform = torchvision.transforms.Compose([
    torchvision.transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10, 
                                       interpolation=InterpolationMode.BILINEAR),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(224)])

train_dataset = torchvision.datasets.CIFAR10('/data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10('/data', train=False, download=True, transform=transform)
# ----------- <End Your code> ---------------

# Define train_loader and test_loader
# ----------- <Your code> ---------------
batch_size = 100

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
def train_model_cifar(classifier, optimizer, epoch):

  classifier.train() 

  for batch_idx, (images, targets) in enumerate(train_loader):

    optimizer.zero_grad()
    images = images.to(device)
    targets = targets.to(device)
    output = classifier(images)
    loss = F.cross_entropy(output, targets) 
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    if batch_idx % 100 == 0: 
      print(f'Epoch {epoch}: [{batch_idx*len(images)}/{len(train_loader.dataset)}] Loss: {loss.item()}')


def test_cifar(classifier, epoch):

  classifier.eval() 

  test_loss = 0
  correct = 0

  with torch.no_grad():
    for images, targets in test_loader:
      images = images.to(device)
      targets = targets.to(device)
      output = classifier(images)
      test_loss += F.cross_entropy(output, targets, reduction='sum').item()
      pred = output.data.max(1, keepdim=True)[1] 
      correct += pred.eq(targets.data.view_as(pred)).sum() 
  
  test_loss /= len(test_loader.dataset)

  print(f'Test result on epoch {epoch}: Avg loss is {test_loss}, Accuracy: {100.*correct/len(test_loader.dataset)}%')



In [9]:
import gc
torch.cuda.empty_cache()
gc.collect()

40

In [16]:
max_epoch = 13

joenet = jn.JoeNet(scale_width=1.0, scale_depth=1.6, output_size=10)
optimizer = optim.Adam(joenet.parameters(), lr=0.01, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#torch.cuda.empty_cache()
joenet.to(device)
losses = []
for epoch in range(1, max_epoch+1):
  train_model_cifar(joenet, optimizer, epoch)
  test_cifar(joenet, epoch)
  scheduler.step()
  for param_group in optimizer.param_groups:
    print(param_group['lr'])
  if (epoch%5==0):
    save_model_all(joenet, 'joe_trained_models', "JoeNet_depth_scale_11_15", epoch)

save_model_all(joenet, 'joe_trained_models', "JoeNet_depth_scale_11_15_final", max_epoch)


print("Exiting")

Epoch 1: [0/50000] Loss: 2.313979148864746
Epoch 1: [10000/50000] Loss: 2.4996044635772705
Epoch 1: [20000/50000] Loss: 2.301668167114258
Epoch 1: [30000/50000] Loss: 2.319676160812378
Epoch 1: [40000/50000] Loss: 2.2128467559814453
Test result on epoch 1: Avg loss is 2.2028085556030272, Accuracy: 14.899999618530273%
0.01
Epoch 2: [0/50000] Loss: 2.1323952674865723
Epoch 2: [10000/50000] Loss: 2.1459860801696777
Epoch 2: [20000/50000] Loss: 2.1340510845184326
Epoch 2: [30000/50000] Loss: 2.1438651084899902
Epoch 2: [40000/50000] Loss: 2.1791064739227295
Test result on epoch 2: Avg loss is 2.2137090957641603, Accuracy: 19.119998931884766%
0.01
Epoch 3: [0/50000] Loss: 2.1250061988830566
Epoch 3: [10000/50000] Loss: 1.9623253345489502
Epoch 3: [20000/50000] Loss: 1.9939978122711182
Epoch 3: [30000/50000] Loss: 2.1419434547424316
Epoch 3: [40000/50000] Loss: 1.9396677017211914
Test result on epoch 3: Avg loss is 1.9481609268188476, Accuracy: 28.149999618530273%
0.01
Epoch 4: [0/50000] Los

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
fig = plt.figure()
ax = plt.axes()

x = np.linspace(0, len(losses), num=len(losses))
plt.title("Loss for B4 over 13 epochs")
plt.xlabel("Training Steps")
plt.ylabel("Cross Entropy Loss");
ax.plot(x, losses)

In [12]:
joenet = jn.JoeNet(scale_width=1.1, scale_depth=1.3, output_size=10)
path = 'joe_trained_models/JoeNet_b1_11_14_epoch_5.pt'
joenet.load_state_dict(torch.load(path))
optimizer = optim.Adam(joenet.parameters(), lr=0.01, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [11]:
import gc

In [13]:
joenet.to(device)
gc.collect()
torch.cuda.empty_cache()
max_epoch = 13

#torch.cuda.empty_cache()
joenet.to(device)
losses = []
for epoch in range(6, max_epoch+1):
  train_model_cifar(joenet, optimizer, epoch)
  test_cifar(joenet, epoch)
  scheduler.step()
  for param_group in optimizer.param_groups:
    print(param_group['lr'])
  if (epoch%5==0):
    save_model_all(joenet, 'joe_trained_models', "JoeNet_b1_11_14", epoch)

save_model_all(joenet, 'joe_trained_models', "JoeNet_b1_11_14_final", max_epoch)


Epoch 6: [0/50000] Loss: 1.8892321586608887
Epoch 6: [20000/50000] Loss: 2.167454957962036
Epoch 6: [40000/50000] Loss: 1.9389538764953613
Test result on epoch 6: Avg loss is 1.9945827026367187, Accuracy: 25.81999969482422%
0.01
Epoch 7: [0/50000] Loss: 1.977778434753418
Epoch 7: [20000/50000] Loss: 1.951385498046875
Epoch 7: [40000/50000] Loss: 1.7900729179382324
Test result on epoch 7: Avg loss is 3.2003136962890624, Accuracy: 20.43000030517578%
0.01
Epoch 8: [0/50000] Loss: 1.8115763664245605
Epoch 8: [20000/50000] Loss: 1.7632575035095215
Epoch 8: [40000/50000] Loss: 1.8249109983444214
Test result on epoch 8: Avg loss is 2.0827800537109376, Accuracy: 32.959999084472656%
0.01
Epoch 9: [0/50000] Loss: 1.6940622329711914
Epoch 9: [20000/50000] Loss: 1.506355881690979
Epoch 9: [40000/50000] Loss: 1.4428455829620361
Test result on epoch 9: Avg loss is 1.6014730255126952, Accuracy: 42.68000030517578%
0.01
Epoch 10: [0/50000] Loss: 1.2980067729949951
Epoch 10: [20000/50000] Loss: 1.395174